In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

%matplotlib inline

In [3]:
# group data by activity
def data_by_activity(X, y, activities):
    # group windows by activity
    grouped=list()
    #return {a:X[y[:,0]==a] for a in activities}
    for i in activities:
        ac = X[Y[:,0]==i]
        grouped.append(ac)
        #print(i,ac.shape)

def load_all_data(directory):
    # Load
    filename="mHealth_subject1.csv"
    df = pd.read_csv(directory+filename)
    df.insert(0, 'id', 1)


    for i in range(9):
        number=str(i+2)
        filename="mHealth_subject"+number+".csv"
    #   print(directory+filename)
        df_subject = pd.read_csv(directory+filename)
        df_subject.insert(0, 'id', i+2)
        df = df.append(df_subject)



    # Cleaning
    df =df.query('label != 0')
    raw = df
    
    # Separate data
    X = df.iloc[:, :24]
    Y = df.iloc[:,24]

    return raw,X,Y


def class_breakdown(data):
    # convert the numpy array into a dataframe
    df = pd.DataFrame(data)
    # group data by the class value and calculate the number of rows
    counts = df.groupby(0).size()
    # retrieve raw rows
    counts = counts.values
    # summarize
    for i in range(len(counts)):
        percent = counts[i] / len(df) * 100
        print('Class=%d, total=%d, percentage=%.3f' % (i + 1, counts[i], percent))


# Method to convert data to series
def to_series(data, off, activity_list, subject_id):
    subject_data = data.query('id==' + str(subject_id))
    series = [[]]
    for activity in activity_list:
        ser = np.asmatrix(subject_data.query("label=="+str(activity)).iloc[:, off]).T
        series=np.append(series,ser)
    return series

In [4]:
directory="Data/MHEALTHDATASET/"

raw,X,y=load_all_data(directory)
raw.head()

,id,acc_chest_x,acc_chest_y,acc_chest_z,elect_sig_1,elect_sig_2,acc_left_ank_x,acc_left_ank_y,acc_left_ank_z,gyr_left_ank_x,...,acc_right_arm_x,acc_right_arm_y,acc_right_arm_z,gyr_right_arm_x,gyr_right_arm_y,gyr_right_arm_z,mag_right_arm_x,mag_right_arm_y,mag_right_arm_z,label
3530,1,-9.8289,0.537270,2.1010,-0.121400,0.163270,2.4495,-9.7277,-0.493140,-0.25788,...,-1.6442,-9.8757,3.43330,-0.90784,-0.57290,0.084052,-3.3505,-11.89000,5.4671,8
3531,1,-10.0630,0.141760,2.0233,-0.192570,0.100470,1.7507,-9.7923,-0.024192,-0.25788,...,-2.5060,-9.7488,2.11670,-0.90784,-0.57290,0.084052,1.9181,-5.67690,-3.1540,8
3532,1,-9.6363,-0.002198,1.6945,-0.226060,0.020931,2.4836,-9.4641,0.124550,-0.25788,...,-2.6872,-9.9677,1.23080,-0.90784,-0.57290,0.084052,4.4654,-2.29000,-6.7470,8
3533,1,-9.5303,0.250110,1.8225,-0.159080,0.129770,2.5743,-9.3353,0.131890,-0.25788,...,-2.6247,-9.7028,0.72319,-0.90784,-0.57290,0.084052,6.4595,-0.37111,-12.5020,8
3534,1,-9.2791,0.317060,1.4391,-0.037677,0.267920,2.8298,-9.2048,0.307640,-0.28200,...,-2.7955,-9.2540,0.59980,-0.91176,-0.58522,0.101290,7.2062,2.48540,-16.4870,8


In [12]:
subject_1=raw.query('id==1')

hr_subject_1=subject_1.iloc[:, [4,5,24]].copy()

hr_subject_1.head()

,elect_sig_1,elect_sig_2,label
3530,-0.121400,0.163270,8
3531,-0.192570,0.100470,8
3532,-0.226060,0.020931,8
3533,-0.159080,0.129770,8
3534,-0.037677,0.267920,8


In [1]:
import numpy as np 
from scipy.special import expit as sigmoid
import torch
from torch import nn


def forget_gate(x, h, Weights_hf, Bias_hf, Weights_xf, Bias_xf, prev_cell_state):
    forget_hidden  = np.dot(Weights_hf, h) + Bias_hf
    forget_eventx  = np.dot(Weights_xf, x) + Bias_xf
    return np.multiply( sigmoid(forget_hidden + forget_eventx), prev_cell_state )

def input_gate(x, h, Weights_hi, Bias_hi, Weights_xi, Bias_xi, Weights_hl, Bias_hl, Weights_xl, Bias_xl):
    ignore_hidden  = np.dot(Weights_hi, h) + Bias_hi
    ignore_eventx  = np.dot(Weights_xi, x) + Bias_xi
    learn_hidden   = np.dot(Weights_hl, h) + Bias_hl
    learn_eventx   = np.dot(Weights_xl, x) + Bias_xl
    return np.multiply( sigmoid(ignore_eventx + ignore_hidden), np.tanh(learn_eventx + learn_hidden) )


def cell_state(forget_gate_output, input_gate_output):
    return forget_gate_output + input_gate_output

  
def output_gate(x, h, Weights_ho, Bias_ho, Weights_xo, Bias_xo, cell_state):
    out_hidden = np.dot(Weights_ho, h) + Bias_ho
    out_eventx = np.dot(Weights_xo, x) + Bias_xo
    return np.multiply( sigmoid(out_eventx + out_hidden), np.tanh(cell_state) )

#Set Parameters for a small LSTM network
input_size  = 2 # size of one 'event', or sample, in our batch of data
hidden_dim  = 3 # 3 cells in the LSTM layer
output_size = 1 # desired model output

def model_output(lstm_output, fc_Weight, fc_Bias):
  '''Takes the LSTM output and transforms it to our desired 
  output size using a final, fully connected layer'''
  return np.dot(fc_Weight, lstm_output) + fc_Bias


#Initialize an PyTorch LSTM for comparison to our Numpy LSTM
class LSTM(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers=1):
        super(LSTM, self).__init__()
        self.hidden_dim=hidden_dim
        #LSTM Layer
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        #Final, fully-connected layer
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = 1
        # get LSTM outputs
        lstm_output, (h,c) = self.lstm(x, hidden)
        # shape output to be (batch_size*seq_length, hidden_dim)
        lstm_output = lstm_output.view(-1, self.hidden_dim)  
        
        # get final output 
        model_output = self.fc(lstm_output)
        
        return model_output, (h,c)
      
torch.manual_seed(5)
torch_lstm = LSTM(input_size = input_size, 
                 hidden_dim = hidden_dim,
                 output_size = output_size,
                 )

state = torch_lstm.state_dict()
#print(state)

#Event (x) Weights and Biases for all gates
Weights_xi = state['lstm.weight_ih_l0'][0:3].numpy()  # shape  [h, x]
Weights_xf = state['lstm.weight_ih_l0'][3:6].numpy()  # shape  [h, x]
Weights_xl = state['lstm.weight_ih_l0'][6:9].numpy()  # shape  [h, x]
Weights_xo = state['lstm.weight_ih_l0'][9:12].numpy() # shape  [h, x]

Bias_xi = state['lstm.bias_ih_l0'][0:3].numpy()  #shape is [h, 1]
Bias_xf = state['lstm.bias_ih_l0'][3:6].numpy()  #shape is [h, 1]
Bias_xl = state['lstm.bias_ih_l0'][6:9].numpy()  #shape is [h, 1]
Bias_xo = state['lstm.bias_ih_l0'][9:12].numpy() #shape is [h, 1]

#Hidden state (h) Weights and Biases for all gates
Weights_hi = state['lstm.weight_hh_l0'][0:3].numpy()  #shape is [h, h]
Weights_hf = state['lstm.weight_hh_l0'][3:6].numpy()  #shape is [h, h]
Weights_hl = state['lstm.weight_hh_l0'][6:9].numpy()  #shape is [h, h]
Weights_ho = state['lstm.weight_hh_l0'][9:12].numpy() #shape is [h, h]

Bias_hi = state['lstm.bias_hh_l0'][0:3].numpy()  #shape is [h, 1]
Bias_hf = state['lstm.bias_hh_l0'][3:6].numpy()  #shape is [h, 1]
Bias_hl = state['lstm.bias_hh_l0'][6:9].numpy()  #shape is [h, 1]
Bias_ho = state['lstm.bias_hh_l0'][9:12].numpy() #shape is [h, 1]

# Final, fully connected layer Weights and Bias
fc_Weight = state['fc.weight'][0].numpy() #shape is [h, output_size]
fc_Bias = state['fc.bias'][0].numpy() #shape is [,output_size]

#--------------------------------------------------------------------
#Simple Time Series Data
data = np.array(
           [[1,1],
            [2,2],
            [3,3]])

#Initialize cell and hidden states with zeroes
h = np.zeros(hidden_dim)
c = np.zeros(hidden_dim)

#Loop through a batch of data, updating the hidden and cell states each time
print('NumPy LSTM Output:')
for eventx in data:
    f = forget_gate(eventx, h, Weights_hf, Bias_hf, Weights_xf, Bias_xf, c)
    i =  input_gate(eventx, h, Weights_hi, Bias_hi, Weights_xi, Bias_xi, 
                Weights_hl, Bias_hl, Weights_xl, Bias_xl)
    c = cell_state(f,i)
    h = output_gate(eventx, h, Weights_ho, Bias_ho, Weights_xo, Bias_xo, c)
    print(model_output(h, fc_Weight, fc_Bias))

NumPy LSTM Output:
-0.3479427319173535
-0.47396493597023465
-0.5263107365187176
